In [1]:
import xarray as xr
import numpy as np

import build_model
import experiments

import dask
import dask.array as da
from dask_jobqueue import PBSCluster
from dask.distributed import Client
dask.config.set(**{'array.slicing.split_large_chunks': True})

2023-07-18 09:56:09.401548: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-18 09:56:09.583930: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-18 09:56:10.466955: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /glade/u/apps/dav/opt/cuda/11.4.0/extras/CUPTI/lib64:/glade/u/apps/dav/opt/cuda/11.4.0/lib64:/glade/u/apps/dav/opt/openmpi/4.1.1/intel/19.1.1/lib:/glade/u/apps/dav/opt/ucx/1.11.0/lib:/glade/u/apps/opt/int

In [2]:
cluster = PBSCluster(
                    cores=1, 
                    memory='10GB', # Amount of memory
                    processes=1, 
                    queue='casper', 
                    local_directory='$TMPDIR', # Use your local directory
                    resource_spec='select=1:ncpus=1:mem=10GB', 
                    project='P06010014', 
                    walltime='10:00:00', 
                    interface='ib0', 
                    )

# Scale up
cluster.scale(5)

client = Client(cluster)
client

/glade/u/home/kjmayer/.conda/envs/tf2-env/lib/python3.10/site-packages/dask_jobqueue/pbs.py:82: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/glade/u/home/kjmayer/.conda/envs/tf2-env/lib/python3.10/site-packages/dask_jobqueue/pbs.py:82: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)


Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/kjmayer/Plot/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/kjmayer/Plot/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.205.16:41990,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/kjmayer/Plot/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [25]:
# client.shutdown()

/glade/u/home/kjmayer/.conda/envs/tf2-env/lib/python3.10/site-packages/dask_jobqueue/pbs.py:82: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)


In [4]:
datapath = '/glade/scratch/kjmayer/DATA/CESM2-LE/HIST/'

atm = 'Z500'
ocn = 'TS'
lnd = 'SOILWATER_10CM'

ens_strt = '1231' # max
atm_finame = atm+'_mem*_strt'+ens_strt+'_1950-2014_check.nc'
lnd_finame = lnd+'_mem*_strt'+ens_strt+'_1950-2014_check.nc'
ocn_finame = ocn+'_mem*_strt'+ens_strt+'_1950-2014_check.nc'

In [5]:
# atmosphere = z500
x1 = xr.open_mfdataset(datapath+'z500'+'/'+atm_finame,
                       concat_dim = 'ens',
                       combine="nested",
                       )[atm]
# land = soil water 10cm
x2 = xr.open_mfdataset(datapath+lnd+'/'+lnd_finame,
                       concat_dim = 'ens',
                       combine="nested",
                       )[lnd]
# ocean = skin temperature
x3 = xr.open_mfdataset(datapath+ocn+'/'+ocn_finame,
                       concat_dim = 'ens',
                       combine="nested",
                       )['__xarray_dataarray_variable__']

## Standardization:
decided to keep seasonal cycle within data - therefore, removed grid point mean & standard deviation across ens members rather than for each doy

!! another option to test: remove seasonal cycle, but add (normalized) doy to flattened layer

In [7]:
x1train = x1[:8].compute()
x1train_mean = x1train.mean('ens').compute()
x1train_std = x1train.std('ens').compute()

# x1train = x1train.stack(alltime=('ens','time'))
# x1train = x1train.compute()
# x1train_mean = x1train.mean('alltime').compute()
# x1train_std = x1train.std('alltime').compute()

del x1train

# Save mean & std:
x1_ds = xr.Dataset(data_vars=dict(
                    trainmean=(["time","lat", "lon"], x1train_mean.data),
                    trainstd=(["time","lat", "lon"], x1train_std.data),
                ),
                coords=dict(
                    time=(["time"], x1train_mean.time.data),
                    lon=(["lon"], x1train_mean.lon.data),
                    lat=(["lat"], x1train_mean.lat.data)
                ),
                attrs=dict(description="z500 members 11-18; start year 1231"),
            )

x1_ds.to_netcdf(datapath+'z500'+'/'+atm+'_meanstd_mem11-18_strt'+ens_strt+'_1950-2014_check.nc')

# x1train_std = (x1train - x1train_mean)/x1train_std

In [8]:
x2train = x2[:8].compute()
x2train_mean = x2train.mean('ens').compute()
x2train_std = x2train.std('ens').compute()

# x2train = x2train.stack(alltime=['ens','time'])
# x2train = x2train.compute()
# x2train_mean = x2train.mean('alltime').compute()
# x2train_std = x2train.std('alltime').compute()

del x2train

# Save mean & std:
x2_ds = xr.Dataset(data_vars=dict(
                    trainmean=(["time","lat", "lon"], x2train_mean.data),
                    trainstd=(["time","lat", "lon"], x2train_std.data),
                ),
                coords=dict(
                    time=(["time"], x2train_mean.time.data),
                    lon=(["lon"], x2train_mean.lon.data),
                    lat=(["lat"], x2train_mean.lat.data)
                ),
                attrs=dict(description="SOILWATER_10CM members 11-18; start year 1231"),
            )

x2_ds.to_netcdf(datapath+lnd+'/'+lnd+'_meanstd_mem11-18_strt'+ens_strt+'_1950-2014_check.nc')

# x2train_std = (x2train - x2train_mean)/x2train_std

In [9]:
x3train = x3[:8].compute()
x3train_mean = x3train.mean('ens').compute()
x3train_std = x3train.std('ens').compute()

# x3train = x3train.stack(alltime=['ens','time'])
# x3train = x3train.compute()
# x3train_mean = x3train.mean('alltime').compute()
# x3train_std = x3train.std('alltime').compute()

del x3train

# Save mean & std:
x3_ds = xr.Dataset(data_vars=dict(
                    trainmean=(["time","lat", "lon"], x3train_mean.data),
                    trainstd=(["time","lat", "lon"], x3train_std.data),
                ),
                coords=dict(
                    time=(["time"], x3train_mean.time.data),
                    lon=(["lon"], x3train_mean.lon.data),
                    lat=(["lat"], x3train_mean.lat.data)
                ),
                attrs=dict(description="TS members 11-18; start year 1231"),
            )

x3_ds.to_netcdf(datapath+ocn+'/'+ocn+'_meanstd_mem11-18_strt'+ens_strt+'_1950-2014_check.nc')

# x3train_std = (x3train - x3train_mean)/x3train_std